In [ ]:
#### BLOCK A




# ===== Header / Config =====
from datetime import date, timedelta

RUN_ID = date.today().isoformat()        # e.g., '2025-09-03'
# Date window for WarSpotting pulls (keep it small at first)
START_DATE = "2025-06-01"
END_DATE   = "2025-09-08"

# Google Drive root (ensure it exists after mounting)
DRIVE_ROOT = "/content/drive/MyDrive/osint_mvp/"
DIRS = {
    "raw":      DRIVE_ROOT + "raw/",
    "raw_ws":   DRIVE_ROOT + "raw/warspotting/",
    "raw_oryx": DRIVE_ROOT + "raw/oryx/",
    "staged":   DRIVE_ROOT + "staged/",
    "ref":      DRIVE_ROOT + "ref/",
    "out":      DRIVE_ROOT + "out/",
    "plots":    DRIVE_ROOT + "plots/",
}

# Focus scope (start small)
# Pull ALL types; filter later in analysis (set to [] or None)
PLATFORM_CLASSES = []  # or: None
TARGET_UNITS = [
    # add a few to start; these appear in your samples
    "236th Artillery Brigade",
    "291st Guards Motor Rifle Regiment",
    "752nd Motor Rifle Regiment",
]

# WarSpotting API
WSP_BASE = "https://ukr.warspotting.net/api/losses/russia"
WSP_USER_AGENT = "primer-osint-etl/0.1 (contact: clinton.brennan.may@gmail.com)"  # REQUIRED to avoid 520s
WSP_REQS_PER_10S = 10  # throttle target

# Oryx CSV mirror (if used); you can add/replace later
ORYX_RAW_GITHUB_TEMPLATE = "https://raw.githubusercontent.com/leedrake5/Russia-Ukraine/master/data/russia_losses_by_equipment.csv"  # example combined file

print("Config OK:", RUN_ID, START_DATE, END_DATE)


Config OK: 2025-09-08 2025-06-01 2025-09-08


In [ ]:
#### BLOCK B



# ===== Setup / Imports / Drive mount =====
!pip -q install pandas requests python-dateutil tqdm

import os, json, time, math, re
import pandas as pd
from dateutil.parser import parse as dtparse
from datetime import datetime, timedelta
from tqdm import tqdm
from google.colab import drive
from IPython.display import display
from datetime import datetime, timezone



# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Ensure folders exist
for p in DIRS.values():
    os.makedirs(p, exist_ok=True)

print("Folders ready under", DRIVE_ROOT)


Mounted at /content/drive
Folders ready under /content/drive/MyDrive/osint_mvp/


In [ ]:
### BLOCK C


# ===== Utils (logging, I/O, DQ checks) =====

def log(msg):
    ts = datetime.now(timezone.utc).isoformat(timespec='seconds')
    print(f"[{ts}] {msg}")

def save_csv(df: pd.DataFrame, path: str, index=False):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    df.to_csv(path, index=index)
    log(f"Saved: {path} ({len(df):,} rows)")

def peek(df: pd.DataFrame, name="df", n=5):
    log(f"Peek {name}: shape={df.shape}")
    display(df.head(n))
    if len(df.columns) <= 40:
        display(df.dtypes)
    print("Null counts:")
    display(df.isna().sum())

def assert_non_empty(df, name="df"):
    assert len(df) > 0, f"{name} is empty"

def assert_no_nulls(df, cols, name="df"):
    nulls = df[cols].isna().sum()
    assert (nulls == 0).all(), f"{name} has NULLs in {list(nulls[nulls>0].index)}"

def assert_enum(df, col, allowed, name="df"):
    bad = set(df[col].dropna().unique()) - set(allowed)
    assert not bad, f"{name}.{col} has unexpected values: {bad}"


In [ ]:
#### BLOCK D



# ===== Expectations (schemas & mappings) =====

# Normalize WarSpotting "type" -> our platform_class
WS_TYPE_TO_CLASS = {
    "Tanks": "MBT",
    "Infantry fighting vehicles": "IFV",
    "Armoured personnel carriers": "APC",
    "Self-propelled artillery": "SPG",
    "Rocket and missile artillery": "MLRS",
    "Towed artillery": "TowedArtillery",
    "Anti-aircraft systems": "AD",
    "Helicopters": "Helicopter",
    "Command posts, communication": "C2",
    "Transport": "Truck",
    "Engineering": "Engineering",
    "Drones": "UAV",
    "Other": "Other",
    # add more as you encounter them
}

# Normalize status
STATUS_NORM = {
    "Destroyed": "destroyed",
    "Damaged": "damaged",
    "Abandoned": "abandoned",
    "Captured": "captured",
}

# Unit aliasing (expand as needed)
UNIT_ALIASES = {
    "291st Guards Motor Rifle Regiment": [
        "291st Guards MRR", "291st GMRR", "291st Motorized Rifle Regiment"
    ],
    "236th Artillery Brigade": [
        "236th ABR", "236th Artillery Brig."
    ],
    "752nd Motor Rifle Regiment": [
        "752nd MRR", "752nd Motorized Rifle Regiment"
    ],
}

# Helper: resolve unit canonical name if matches any alias (simple pass)
def canonical_unit(u: str):
    if not isinstance(u, str) or not u.strip():
        return None
    u_stripped = u.strip()
    for canon, aliases in UNIT_ALIASES.items():
        if u_stripped == canon or any(u_stripped == a for a in aliases):
            return canon
    return u_stripped  # fallback to original text


In [ ]:
#### BLOCK E



# ===== Fetch – WarSpotting (raw → bronze) =====

import random, time, requests

def ws_get(url, max_retries=6, base_backoff=1.5):
    """GET with UA, handles 429/5xx using exponential backoff + jitter."""
    headers = {"User-Agent": WSP_USER_AGENT}
    for attempt in range(max_retries):
        r = requests.get(url, headers=headers, timeout=20)
        if r.status_code == 200:
            return r.json()

        # Respect Retry-After if present, else exponential backoff
        if r.status_code in {429, 502, 503, 504, 520}:
            retry_after = r.headers.get("Retry-After")
            if retry_after and retry_after.isdigit():
                wait = float(retry_after)
            else:
                wait = base_backoff * (2 ** attempt)
            wait += random.uniform(0, 0.6)  # jitter
            log(f"{r.status_code} on {url} — sleeping {wait:.1f}s (attempt {attempt+1}/{max_retries})")
            time.sleep(wait)
            continue

        # Non-retryable
        raise RuntimeError(f"Failed GET {url} -> {r.status_code}: {r.text[:200]}")
    raise RuntimeError(f"Exceeded retries for {url} (last={r.status_code})")


def daterange(start_iso, end_iso):
    d0 = datetime.fromisoformat(start_iso)
    d1 = datetime.fromisoformat(end_iso)
    d = d0
    while d <= d1:
        yield d.date().isoformat()
        d += timedelta(days=1)

def polite_sleep(base=1.2, jitter=0.25):
    import time
    time.sleep(base + random.uniform(-jitter, jitter))

def fetch_ws_daily_jsonl(start_iso, end_iso):
    """Pull daily pages; write JSONL per day (bronze). Skips days already present."""
    for day in daterange(start_iso, end_iso):
        day_out = os.path.join(DIRS["raw_ws"], f"losses_{day}.jsonl")
        if os.path.exists(day_out):  # idempotent skip
            log(f"Raw exists, skipping: {day_out}")
            continue
        page = 1
        wrote_any = False
        with open(day_out, "w", encoding="utf-8") as f:
            while True:
                url = f"{WSP_BASE}/{day}/{page}"
                data = ws_get(url)
                losses = data.get("losses", [])
                if not losses:
                    break
                for row in losses:
                    row["_source_url"] = url
                    row["_retrieved_at"] = datetime.now(timezone.utc).isoformat(timespec="seconds")
                    f.write(json.dumps(row, ensure_ascii=False) + "\n")
                    wrote_any = True
                page += 1
                polite_sleep()
        log(f"Wrote raw JSONL for {day}: {day_out} (has_data={wrote_any})")
        # checkpoint each day
        with open(os.path.join(DIRS["out"], "checkpoint.json"), "w") as ck:
            json.dump({"last_completed_day": day}, ck)


# Run the pull (start small; expand the window later)
fetch_ws_daily_jsonl(START_DATE, END_DATE)


[2025-09-08T19:49:57+00:00] Raw exists, skipping: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-01.jsonl
[2025-09-08T19:49:57+00:00] Raw exists, skipping: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-02.jsonl
[2025-09-08T19:49:57+00:00] Raw exists, skipping: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-03.jsonl
[2025-09-08T19:49:57+00:00] Raw exists, skipping: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-04.jsonl
[2025-09-08T19:49:57+00:00] Raw exists, skipping: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-05.jsonl
[2025-09-08T19:50:00+00:00] Wrote raw JSONL for 2025-06-06: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-06.jsonl (has_data=True)
[2025-09-08T19:50:04+00:00] Wrote raw JSONL for 2025-06-07: /content/drive/MyDrive/osint_mvp/raw/warspotting/losses_2025-06-07.jsonl (has_data=True)
[2025-09-08T19:50:07+00:00] Wrote raw JSONL for 2025-06-08: /content/drive/MyDrive/osin

In [ ]:
# ===== Backfill controller (optional) =====
# Purpose: backfill in monthly chunks; respects skip-if-exists
from calendar import monthrange

def month_chunks(start_iso, end_iso):
    s = datetime.fromisoformat(start_iso).date()
    e = datetime.fromisoformat(end_iso).date()
    cur = date(s.year, s.month, 1)
    while cur <= e:
        y, m = cur.year, cur.month
        last_day = monthrange(y, m)[1]
        chunk_start = max(cur, s)
        chunk_end = min(date(y, m, last_day), e)
        yield chunk_start.isoformat(), chunk_end.isoformat()
        # next month
        if m == 12: cur = date(y+1, 1, 1)
        else:       cur = date(y, m+1, 1)

# Example: backfill safely (comment/uncomment to use)
# for cs, ce in month_chunks("2025-01-01", END_DATE):
#     log(f"Backfilling {cs} → {ce}")
#     fetch_ws_daily_jsonl(cs, ce)


In [ ]:
# ===== Normalize – WarSpotting (bronze → silver) =====
# What/Why:
# - Load bronze JSONL files for the configured date window (START_DATE..END_DATE)
# - Normalize to a tidy schema with consistent columns/enums
# - Save a single staged CSV for downstream analytics
# Notes:
# - Idempotent: re-running overwrites the same staged file for this RUN_ID
# - Requires Blocks C/D (utils + mappings) to be run first

def load_ws_bronze_to_df():
    rows = []
    start_d = pd.to_datetime(START_DATE).date()
    end_d = pd.to_datetime(END_DATE).date()
    for fn in sorted(os.listdir(DIRS["raw_ws"])):
        if not fn.startswith("losses_") or not fn.endswith(".jsonl"):
            continue
        # expect losses_YYYY-MM-DD.jsonl
        try:
            fdate = pd.to_datetime(fn.split("_")[1].split(".")[0]).date()
        except Exception:
            continue
        if not (start_d <= fdate <= end_d):
            continue
        with open(os.path.join(DIRS["raw_ws"], fn), "r", encoding="utf-8") as f:
            for line in f:
                try:
                    rows.append(json.loads(line))
                except:
                    pass
    return pd.DataFrame(rows)

def normalize_ws(df_raw: pd.DataFrame) -> pd.DataFrame:
    if df_raw is None or df_raw.empty:
        return pd.DataFrame(columns=[
            "date","side","type_raw","model","platform_class","status",
            "location","lat","lon","unit_text","unit_canonical",
            "tags","source","source_url","retrieved_at","count"
        ])
    dd = df_raw.copy()
    dd["date"] = pd.to_datetime(dd.get("date"), errors="coerce").dt.date
    dd["side"] = dd.get("lost_by").fillna("Russia").map({"Russia":"RU","Ukraine":"UA"}).fillna("RU")
    dd["type_raw"] = dd.get("type")
    dd["model"] = dd.get("model")
    # normalize platform_class with fallback to type_raw
    dd["platform_class"] = dd["type_raw"].map(WS_TYPE_TO_CLASS)
    dd.loc[dd["platform_class"].isna(), "platform_class"] = dd["type_raw"]
    dd["status"] = dd.get("status").map(STATUS_NORM).fillna(dd.get("status"))
    dd["location"] = dd.get("nearest_location")

    geo = dd.get("geo").fillna("").astype(str).str.split(",", n=1, expand=True)
    dd["lat"] = pd.to_numeric(geo[0], errors="coerce")
    dd["lon"] = pd.to_numeric(geo[1], errors="coerce")

    dd["unit_text"] = dd.get("unit")
    dd["unit_canonical"] = dd["unit_text"].apply(canonical_unit)
    dd["tags"] = dd.get("tags")
    dd["source"] = "warspotting"
    dd["source_url"] = dd.get("_source_url", "")
    dd["retrieved_at"] = dd.get("_retrieved_at", datetime.now(timezone.utc).isoformat(timespec="seconds"))
    dd["count"] = 1

    cols = ["date","side","type_raw","model","platform_class","status","location","lat","lon",
            "unit_text","unit_canonical","tags","source","source_url","retrieved_at","count"]
    dd = dd[cols].dropna(subset=["date"])
    return dd


# ---- Build silver table ----
ws_raw = load_ws_bronze_to_df()
log(f"Loaded WarSpotting bronze rows: {len(ws_raw):,}")

ws_silver = normalize_ws(ws_raw)

# Filter MVP scope
ws_silver = ws_silver.query("side == 'RU'")
if PLATFORM_CLASSES:
    ws_silver = ws_silver[ws_silver["platform_class"].isin(PLATFORM_CLASSES)]

# Save staged
ws_path = os.path.join(DIRS["staged"], f"warspotting_norm_{RUN_ID}.csv")
save_csv(ws_silver, ws_path)
peek(ws_silver, "ws_silver")

unknown_types = set(ws_silver["type_raw"].unique()) - set(WS_TYPE_TO_CLASS.keys())
if unknown_types:
    log(f"Unmapped WarSpotting types encountered (mapped to themselves): {unknown_types}")


[2025-09-08T19:59:24+00:00] Loaded WarSpotting bronze rows: 493
[2025-09-08T19:59:24+00:00] Saved: /content/drive/MyDrive/osint_mvp/staged/warspotting_norm_2025-09-08.csv (493 rows)
[2025-09-08T19:59:24+00:00] Peek ws_silver: shape=(493, 16)


,date,side,type_raw,model,platform_class,status,location,lat,lon,unit_text,unit_canonical,tags,source,source_url,retrieved_at,count
0,2025-06-01,RU,Infantry mobility vehicles,Asteys-7020 'Patrol',Infantry mobility vehicles,destroyed,"Kostiantynivka, Sumy raion",51.170105,34.814331,None,None,None,warspotting,https://ukr.warspotting.net/api/losses/russia/...,2025-09-08T19:43:53+00:00,1
1,2025-06-01,RU,Airplanes,An-12BK transport aircraft,Airplanes,destroyed,"Olenya airbase, Olenegorsk",68.146846,33.451296,40th Mixed Aviation Regiment,40th Mixed Aviation Regiment,None,warspotting,https://ukr.warspotting.net/api/losses/russia/...,2025-09-08T19:43:53+00:00,1
2,2025-06-01,RU,Airplanes,Tu-95MS strategic bomber,Airplanes,destroyed,"Olenya airbase, Olenegorsk",68.145870,33.450454,43rd Center for Combat Employment and Retraini...,43rd Center for Combat Employment and Retraini...,None,warspotting,https://ukr.warspotting.net/api/losses/russia/...,2025-09-08T19:43:53+00:00,1
3,2025-06-01,RU,Airplanes,Tu-95MS strategic bomber,Airplanes,destroyed,"Olenya airbase, Olenegorsk",68.144802,33.449353,184th Heavy Bomber Aviation Regiment,184th Heavy Bomber Aviation Regiment,None,warspotting,https://ukr.warspotting.net/api/losses/russia/...,2025-09-08T19:43:53+00:00,1
4,2025-06-01,RU,Airplanes,Tu-95MS strategic bomber,Airplanes,destroyed,"Olenya airbase, Olenegorsk",68.143713,33.448580,184th Heavy Bomber Aviation Regiment,184th Heavy Bomber Aviation Regiment,None,warspotting,https://ukr.warspotting.net/api/losses/russia/...,2025-09-08T19:43:53+00:00,1


,0
date,object
side,object
type_raw,object
model,object
platform_class,object
status,object
location,object
lat,float64
lon,float64
unit_text,object


Null counts:


,0
date,0
side,0
type_raw,0
model,0
platform_class,0
status,0
location,74
lat,238
lon,238
unit_text,448


[2025-09-08T19:59:24+00:00] Unmapped WarSpotting types encountered (mapped to themselves): {'Infantry mobility vehicles', 'Ambulances, medical vehicles', 'Airplanes', 'Radars, jammers'}


In [ ]:
#### BLOCK G (OPTIONAL)

# ===== (Optional) Oryx fetch – CSV mirror (raw → bronze) =====
# For MVP, pull a single combined CSV mirror; later you can switch to date-specific files.

import requests, io

def fetch_oryx_combined_csv():
    r = requests.get(ORYX_RAW_GITHUB_TEMPLATE, timeout=30)
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text))

try:
    oryx_bronze = fetch_oryx_combined_csv()
    oryx_raw_path = os.path.join(DIRS["raw_oryx"], f"oryx_combined_{RUN_ID}.csv")
    save_csv(oryx_bronze, oryx_raw_path)
    peek(oryx_bronze, "oryx_bronze", n=3)
except Exception as e:
    log(f"Oryx fetch skipped or failed: {e}")
    oryx_bronze = pd.DataFrame()


In [ ]:
#### BLOCK H (OPTIONAL)

# ============================================================
# Block H — Normalize Oryx (bronze → silver)
# Purpose:
#   - Take the Oryx mirror CSV (if fetched) and align it to the
#     same normalized schema as WarSpotting so we can UNION them.
#   - Keep scope tight: RU side only; focus on PLATFORM_CLASSES (MBT/IFV for MVP).
# Outputs:
#   - /staged/oryx_norm_{RUN_ID}.csv  (row-level tidy)
# Notes:
#   - Different mirrors have different column names. Map what you have to:
#       date | side | model | platform_class | status | location |
#       lat | lon | unit_text | unit_canonical | tags | source | source_url | retrieved_at | count
# ============================================================

def normalize_oryx(df_raw: pd.DataFrame) -> pd.DataFrame:
    if df_raw is None or df_raw.empty:
        return pd.DataFrame(columns=[
            "date","side","model","platform_class","status","location","lat","lon",
            "unit_text","unit_canonical","tags","source","source_url","retrieved_at","count"
        ])

    dd = df_raw.copy()

    # --- Heuristic column mapping (adapt as needed for your mirror) ---
    # Try common column names; adjust these if your mirror uses different labels.
    candidate_cols = {
        "date":          ["date", "Date"],
        "side":          ["side", "Belligerent", "Lost by"],
        "model":         ["model", "Equipment", "Model"],
        "type":          ["type", "Type", "Equipment class", "Category"],
        "status":        ["status", "Loss status", "Status"],
        "location":      ["location", "Location"],
        "lat":           ["lat", "latitude", "Lat"],
        "lon":           ["lon", "longitude", "Lon"],
        "unit_text":     ["unit", "Unit", "Sub‑unit"],
        "tags":          ["tags", "Tags", "Notes"],
        "source_url":    ["source", "Source", "Source link", "URL"]
    }

    def pick(colnames):
        for c in colnames:
            if c in dd.columns:
                return dd[c]
        return None

    # Build normalized frame
    out = pd.DataFrame()
    out["retrieved_at"] = datetime.now(timezone.utc).isoformat(timespec="seconds")
    side_raw = pick(candidate_cols["side"])
    if side_raw is None:
        side_raw = "Russia"
    out["side"] = side_raw.map({"Russia":"RU","Ukraine":"UA"}).fillna("RU") if hasattr(side_raw, "map") else "RU"
    out["model"] = pick(candidate_cols["model"])

    # Map type → our platform_class using the same dict we used for WarSpotting where possible
    type_series = pick(candidate_cols["type"])
    out["platform_class"] = type_series.map(WS_TYPE_TO_CLASS).fillna(type_series)

    # Normalize status to our lower-case enum
    status_series = pick(candidate_cols["status"])
    if status_series is not None:
        out["status"] = status_series.map(STATUS_NORM).fillna(status_series)
    else:
        out["status"] = None

    out["location"] = pick(candidate_cols["location"])

    # Geo (coerce numeric)
    out["lat"] = pd.to_numeric(pick(candidate_cols["lat"]), errors="coerce")
    out["lon"] = pd.to_numeric(pick(candidate_cols["lon"]), errors="coerce")

    out["unit_text"] = pick(candidate_cols["unit_text"])
    out["unit_canonical"] = out["unit_text"].apply(canonical_unit) if "unit_text" in out else None
    out["tags"] = pick(candidate_cols["tags"])

    # Provenance
    out["source"] = "oryx"
    out["source_url"] = pick(candidate_cols["source_url"])
    out["retrieved_at"] = datetime.utcnow().isoformat(timespec="seconds")+"Z"
    out["count"] = 1

    # Filter MVP scope: RU + selected classes + non-null dates
    out = out.dropna(subset=["date"])
    out = out[out["side"] == "RU"]
    if PLATFORM_CLASSES:
        out = out[out["platform_class"].isin(PLATFORM_CLASSES)]

    # Save staged if not empty
    oryx_silver = out.copy()
    if not oryx_silver.empty:
        oryx_path = os.path.join(DIRS["staged"], f"oryx_norm_{RUN_ID}.csv")
        save_csv(oryx_silver, oryx_path)
        peek(oryx_silver, "oryx_silver", n=5)
    else:
        log("Oryx silver is empty; skipping save.")


In [ ]:
#### BLOCK I (OPTIONAL)

# ============================================================
# Block I — Data Quality checks (silver tables)
# Purpose:
#   - Validate core columns and enums for warspotting and (optional) oryx.
#   - Check date coverage within our window.
# Outputs:
#   - Prints to console; raises AssertionError on violation.
# ============================================================

# Re-load silver tables from disk so checks run on persisted artifacts
ws_silver_path = os.path.join(DIRS["staged"], f"warspotting_norm_{RUN_ID}.csv")
ws_silver = pd.read_csv(ws_silver_path, parse_dates=["date"])

# Oryx is optional
oryx_silver_path = os.path.join(DIRS["staged"], f"oryx_norm_{RUN_ID}.csv")
oryx_silver = None
if os.path.exists(oryx_silver_path):
    oryx_silver = pd.read_csv(oryx_silver_path, parse_dates=["date"])

# --- Required columns we expect everywhere ---
REQUIRED_COLS = ["date","side","model","platform_class","status","location","unit_text","unit_canonical","source","count"]

# WarSpotting checks
assert_non_empty(ws_silver, "ws_silver")
missing_ws = set(REQUIRED_COLS) - set(ws_silver.columns)
assert not missing_ws, f"ws_silver missing columns: {missing_ws}"

# Enums
assert_enum(ws_silver, "side", {"RU","UA"}, "ws_silver")
assert_enum(ws_silver, "status", {"destroyed","damaged","abandoned","captured", None, pd.NA, float("nan")}, "ws_silver")
if PLATFORM_CLASSES:
    assert set(ws_silver["platform_class"].dropna().unique()).issubset(set(PLATFORM_CLASSES)), \
        f"ws_silver has unexpected platform_class outside {PLATFORM_CLASSES}"

# Date window sanity
min_d, max_d = ws_silver["date"].min().date(), ws_silver["date"].max().date()
log(f"WarSpotting date coverage: {min_d} → {max_d} (requested {START_DATE} → {END_DATE})")

# Oryx checks (if present)
if oryx_silver is not None and not oryx_silver.empty:
    assert_non_empty(oryx_silver, "oryx_silver")
    missing_ox = set(REQUIRED_COLS) - set(oryx_silver.columns)
    assert not missing_ox, f"oryx_silver missing columns: {missing_ox}"
    if PLATFORM_CLASSES:
        assert set(oryx_silver["platform_class"].dropna().unique()).issubset(set(PLATFORM_CLASSES)), \
            f"oryx_silver has unexpected platform_class outside {PLATFORM_CLASSES}"
    min_do, max_do = oryx_silver["date"].min().date(), oryx_silver["date"].max().date()
    log(f"Oryx date coverage: {min_do} → {max_do}")
else:
    log("Oryx silver not present (skipped or empty) — OK for MVP.")

[2025-09-08T20:04:02+00:00] WarSpotting date coverage: 2025-06-01 → 2025-09-07 (requested 2025-06-01 → 2025-09-08)
[2025-09-08T20:04:02+00:00] Oryx silver not present (skipped or empty) — OK for MVP.


In [ ]:
#### BLOCK J

# ============================================================
# Block J — Gold outputs (union + rollups)
# Purpose:
#   - UNION warspotting + oryx (if present) into a single tidy table.
#   - Produce daily rollups and cumulative series by platform_class & source.
# Outputs:
#   - /out/merged_losses_{RUN_ID}.csv         (row-level unioned)
#   - /out/merged_daily_class_{RUN_ID}.csv    (daily totals by class & source)
# ============================================================

# Build union
parts = [ws_silver]
if oryx_silver is not None and not oryx_silver.empty:
    parts.append(oryx_silver)

merged = pd.concat(parts, ignore_index=True)
merged["date"] = pd.to_datetime(merged["date"]).dt.date
start_d = pd.to_datetime(START_DATE).date()
end_d = pd.to_datetime(END_DATE).date()
merged = merged[(merged["date"] >= start_d) & (merged["date"] <= end_d)]

# Keep only requested classes & RU
merged = merged[merged["side"] == "RU"]
if PLATFORM_CLASSES:
    merged = merged[merged["platform_class"].isin(PLATFORM_CLASSES)]

# Persist row-level gold
merged_out = os.path.join(DIRS["out"], f"merged_losses_{RUN_ID}.csv")
save_csv(merged, merged_out)

# Daily rollups (per source & class)
# Note: Ensure date is a date (not datetime) for grouping readability
merged["date"] = pd.to_datetime(merged["date"]).dt.date
daily = (merged
         .groupby(["date","platform_class","source"], as_index=False)["count"]
         .sum()
         .rename(columns={"count":"n_daily"}))

# Cumulative per (platform_class, source) over time
daily = daily.sort_values(["platform_class","source","date"])
daily["n_cumulative"] = (daily
                         .groupby(["platform_class","source"])["n_daily"]
                         .cumsum())

daily_out = os.path.join(DIRS["out"], f"merged_daily_class_{RUN_ID}.csv")
save_csv(daily, daily_out)

# Type catalog for taxonomy visibility
type_catalog = (merged
    .groupby(["type_raw","platform_class","source"], as_index=False)["count"].sum()
    .sort_values("count", ascending=False))
typecat_out = os.path.join(DIRS["out"], f"type_catalog_{RUN_ID}.csv")
save_csv(type_catalog, typecat_out)


peek(daily, "daily_rollups", n=10)


[2025-09-08T20:05:41+00:00] Saved: /content/drive/MyDrive/osint_mvp/out/merged_losses_2025-09-08.csv (493 rows)
[2025-09-08T20:05:41+00:00] Saved: /content/drive/MyDrive/osint_mvp/out/merged_daily_class_2025-09-08.csv (294 rows)
[2025-09-08T20:05:41+00:00] Saved: /content/drive/MyDrive/osint_mvp/out/type_catalog_2025-09-08.csv (16 rows)
[2025-09-08T20:05:41+00:00] Peek daily_rollups: shape=(294, 5)


,date,platform_class,source,n_daily,n_cumulative
2,2025-06-02,AD,warspotting,1,1
13,2025-06-06,AD,warspotting,1,2
41,2025-06-14,AD,warspotting,1,3
84,2025-06-26,AD,warspotting,1,4
89,2025-06-28,AD,warspotting,1,5
103,2025-07-02,AD,warspotting,1,6
119,2025-07-08,AD,warspotting,1,7
124,2025-07-10,AD,warspotting,1,8
245,2025-08-20,AD,warspotting,1,9
265,2025-08-27,AD,warspotting,1,10


,0
date,object
platform_class,object
source,object
n_daily,int64
n_cumulative,int64


Null counts:


,0
date,0
platform_class,0
source,0
n_daily,0
n_cumulative,0


In [ ]:
#### BLOCK K

# ============================================================
# Block K — Plots (save to Drive)
# Purpose:
#   - Visualize the cumulative and daily losses for the chosen classes & sources.
#   - Save PNGs into /plots so you can drop them into slides quickly.
# ============================================================

import matplotlib.pyplot as plt

def plot_cumulative_by_class(daily_df: pd.DataFrame, platform_class: str, filename: str):
    dfc = daily_df[daily_df["platform_class"] == platform_class].copy()
    if dfc.empty:
        log(f"No data to plot for {platform_class}")
        return
    plt.figure(figsize=(10, 4))
    for src, grp in dfc.groupby("source"):
        plt.plot(grp["date"], grp["n_cumulative"], label=src)
    plt.title(f"Cumulative losses — {platform_class}")
    plt.xlabel("Date"); plt.ylabel("Count (cumulative)")
    plt.legend(); plt.tight_layout()
    outpath = os.path.join(DIRS["plots"], filename)
    plt.savefig(outpath, dpi=160)
    plt.close()
    log(f"Saved plot: {outpath}")

def plot_daily_last_n(daily_df: pd.DataFrame, platform_class: str, days: int, filename: str):
    cutoff = daily_df["date"].max() - pd.Timedelta(days=days-1)
    dfd = daily_df[(daily_df["platform_class"] == platform_class) & (daily_df["date"] >= cutoff)].copy()
    if dfd.empty:
        log(f"No data to plot (last {days}d) for {platform_class}")
        return
    plt.figure(figsize=(10, 4))
    for src, grp in dfd.groupby("source"):
        # aggregate by date in case multiple rows per day exist per source
        grp_ag = grp.groupby("date", as_index=False)["n_daily"].sum()
        plt.plot(grp_ag["date"], grp_ag["n_daily"], label=src)
    plt.title(f"Daily losses (last {days} days) — {platform_class}")
    plt.xlabel("Date"); plt.ylabel("Count per day")
    plt.legend(); plt.tight_layout()
    outpath = os.path.join(DIRS["plots"], filename)
    plt.savefig(outpath, dpi=160)
    plt.close()
    log(f"Saved plot: {outpath}")

# Pick classes dynamically from the data (top 6 by total volume)
top_classes = (daily.groupby("platform_class")["n_cumulative"].max()
                     .sort_values(ascending=False).head(6).index.tolist())

for cls in top_classes:
    plot_cumulative_by_class(daily, cls, filename=f"cum_losses_{cls}_{RUN_ID}.png")
    plot_daily_last_n(daily, cls, days=14, filename=f"daily_last14_{cls}_{RUN_ID}.png")



[2025-09-08T20:05:54+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/cum_losses_IFV_2025-09-08.png
[2025-09-08T20:05:55+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/daily_last14_IFV_2025-09-08.png
[2025-09-08T20:05:55+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/cum_losses_Truck_2025-09-08.png
[2025-09-08T20:05:55+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/daily_last14_Truck_2025-09-08.png
[2025-09-08T20:05:56+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/cum_losses_MBT_2025-09-08.png
[2025-09-08T20:05:56+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/daily_last14_MBT_2025-09-08.png
[2025-09-08T20:05:56+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/cum_losses_TowedArtillery_2025-09-08.png
[2025-09-08T20:05:56+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/daily_last14_TowedArtillery_2025-09-08.png
[2025-09-08T20:05:57+00:00] Saved plot: /content/drive/MyDrive/osint_mvp/plots/cum_losses_MLRS

In [ ]:
# ============================================================
# Block L — Run Manifest / README
# ============================================================
def write_run_manifest():
    import glob
    lines = []
    lines.append(f"OSINT MVP — Run Manifest")
    lines.append(f"RUN_ID: {RUN_ID}")
    lines.append(f"Date window (WarSpotting): {START_DATE} → {END_DATE}")
    lines.append("")
    lines.append("Silver tables:")
    ws_silver_path = os.path.join(DIRS['staged'], f"warspotting_norm_{RUN_ID}.csv")
    oryx_silver_path = os.path.join(DIRS['staged'], f"oryx_norm_{RUN_ID}.csv")
    ws_rows = pd.read_csv(ws_silver_path).shape[0] if os.path.exists(ws_silver_path) else 0
    ox_rows = pd.read_csv(oryx_silver_path).shape[0] if os.path.exists(oryx_silver_path) else 0
    lines.append(f"  - warspotting_norm_{RUN_ID}.csv rows: {ws_rows:,}")
    lines.append(f"  - oryx_norm_{RUN_ID}.csv rows: {ox_rows:,}" if ox_rows else "  - oryx_norm: (not present)")

    merged_out = os.path.join(DIRS["out"], f"merged_losses_{RUN_ID}.csv")
    daily_out  = os.path.join(DIRS["out"], f"merged_daily_class_{RUN_ID}.csv")
    merged_rows = pd.read_csv(merged_out).shape[0] if os.path.exists(merged_out) else 0
    daily_rows  = pd.read_csv(daily_out).shape[0]  if os.path.exists(daily_out)  else 0

    lines.append("")
    lines.append("Gold outputs:")
    lines.append(f"  - merged_losses_{RUN_ID}.csv rows: {merged_rows:,}")
    lines.append(f"  - merged_daily_class_{RUN_ID}.csv rows: {daily_rows:,}")

    # Coverage (WarSpotting silver)
    try:
        if os.path.exists(ws_silver_path):
            ws_tmp = pd.read_csv(ws_silver_path)
            tot = len(ws_tmp)
            geo = (ws_tmp["lat"].notna() & ws_tmp["lon"].notna()).sum()
            unit = ws_tmp["unit_text"].notna().sum()
            lines.append("")
            lines.append("Coverage (WarSpotting silver):")
            lines.append(f"  - rows: {tot:,}")
            lines.append(f"  - with coordinates: {geo:,} ({(geo/max(tot,1))*100:.1f}%)")
            lines.append(f"  - with unit text: {unit:,} ({(unit/max(tot,1))*100:.1f}%)")
    except Exception as e:
        lines.append(f"(coverage calc skipped: {e})")

    # Type catalog presence
    tc_path = os.path.join(DIRS["out"], f"type_catalog_{RUN_ID}.csv")
    if os.path.exists(tc_path):
        lines.append("")
        lines.append("Taxonomy:")
        lines.append(f"  - type_catalog_{RUN_ID}.csv")

    # Plots
    lines.append("")
    lines.append("Plots:")
    plot_files = [fn for fn in sorted(os.listdir(DIRS["plots"])) if fn.endswith(".png") and RUN_ID in fn]
    if plot_files:
        for fn in plot_files:
            lines.append(f"  - {fn}")
    else:
        lines.append("  - (none yet)")

    # Scope & caveats
    lines.append("")
    lines.append("Scope & caveats:")
    lines.append(f"  - side=RU only; classes={PLATFORM_CLASSES}")
    lines.append("  - Visual confirmations undercount true losses")
    lines.append("  - WarSpotting date filters used; Oryx is mirror-normalized if present")
    lines.append("  - All times/dates treated as UTC dates (no hour precision)")

    out_path = os.path.join(DIRS["out"], f"README_{RUN_ID}.txt")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    log(f"Wrote manifest: {out_path}")

write_run_manifest()


[2025-09-08T20:06:05+00:00] Wrote manifest: /content/drive/MyDrive/osint_mvp/out/README_2025-09-08.txt


In [ ]:
# ============================================================
# Block M — ISW snapshot (load → normalize → compare)
# Purpose:
#   - Load your ISW reference snapshot (CSV) with equipment estimates.
#   - Normalize to platform_class rows compatible with our classes.
#   - Produce a simple comparison table vs cumulative OSINT losses.
# Inputs (CSV expected minimal columns):
#   - as_of_date, platform_class, est_equip, source_url, notes
# Outputs:
#   - /out/isw_norm_{RUN_ID}.csv
#   - /out/losses_vs_isw_{RUN_ID}.csv
#   - Optional: small bar chart saved to /plots
# ============================================================

# 1) Where is the ISW CSV?
ISW_REF_PATH = os.path.join(DIRS["ref"], f"isw_force_snapshot_{RUN_ID}.csv")

# Example of the expected minimal structure (you can edit your CSV to match):
# as_of_date,platform_class,est_equip,source_url,notes
# 2025-09-01,MBT,2100,https://...,ISW sheet exported 2025-09-01
# 2025-09-01,IFV,3500,https://...,...

# 2) Load and normalize ISW
def load_isw_snapshot(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        log(f"ISW file not found at {path}. Please add it, then rerun this block.")
        return pd.DataFrame(columns=["as_of_date","platform_class","est_equip","source_url","notes"])
    df = pd.read_csv(path)
    # Normalize columns
    df = df.rename(columns={
        "Platform Class":"platform_class", "platform":"platform_class",
        "Equipment":"platform_class", "Estimated Equipment":"est_equip",
        "as_of":"as_of_date", "As of":"as_of_date"
    })
    # Keep only needed cols
    needed = ["as_of_date","platform_class","est_equip","source_url","notes"]
    for col in needed:
        if col not in df.columns:
            df[col] = None
    # Coerce
    df["as_of_date"] = pd.to_datetime(df["as_of_date"], errors="coerce").dt.date
    df["platform_class"] = df["platform_class"].astype(str).str.strip()
    df["est_equip"] = pd.to_numeric(df["est_equip"], errors="coerce")
    # Filter to our current classes
    if PLATFORM_CLASSES:
        df = df[df["platform_class"].isin(PLATFORM_CLASSES)]
    return df[needed]

isw = load_isw_snapshot(ISW_REF_PATH)
if not isw.empty:
    isw_out = os.path.join(DIRS["out"], f"isw_norm_{RUN_ID}.csv")
    save_csv(isw, isw_out)
    peek(isw, "isw_norm", n=5)
else:
    log("ISW snapshot not available or empty — comparison will be skipped.")

# 3) Compute a simple comparison: cumulative OSINT losses vs ISW est_equip
def build_losses_vs_isw(isw_df: pd.DataFrame, daily_df: pd.DataFrame) -> pd.DataFrame:
    if isw_df.empty or daily_df.empty:
        return pd.DataFrame(columns=["platform_class","isw_as_of","isw_est_equip","osint_cum_losses","losses_over_est_pct"])
    # latest cumulative per class (sum across sources)
    cum_latest = (daily_df
                  .sort_values("date")
                  .groupby(["platform_class","date"], as_index=False)["n_cumulative"].sum())
    # take last date per class
    idx = cum_latest.groupby("platform_class")["date"].transform("max") == cum_latest["date"]
    cum_latest = cum_latest[idx][["platform_class","date","n_cumulative"]].rename(columns={"date":"osint_as_of"})
    # join to ISW (assume single as_of per class; if multiple, take most recent)
    isw_latest = (isw_df
                  .sort_values("as_of_date")
                  .groupby("platform_class", as_index=False)
                  .last()
                  .rename(columns={"as_of_date":"isw_as_of","est_equip":"isw_est_equip"}))
    out = pd.merge(isw_latest, cum_latest, on="platform_class", how="inner")
    out["osint_cum_losses"] = out["n_cumulative"]
    out["losses_over_est_pct"] = (out["osint_cum_losses"] / out["isw_est_equip"]) * 100.0
    out = out[["platform_class","isw_as_of","isw_est_equip","osint_as_of","osint_cum_losses","losses_over_est_pct"]]
    return out

losses_vs_isw = build_losses_vs_isw(isw, daily) if not isw.empty else pd.DataFrame()
if not losses_vs_isw.empty:
    comp_out = os.path.join(DIRS["out"], f"losses_vs_isw_{RUN_ID}.csv")
    save_csv(losses_vs_isw, comp_out)
    peek(losses_vs_isw, "losses_vs_isw", n=10)

    # tiny bar chart per class
    import matplotlib.pyplot as plt
    plt.figure(figsize=(6,4))
    x = losses_vs_isw["platform_class"]
    y = losses_vs_isw["losses_over_est_pct"]
    plt.bar(x, y)
    plt.title("OSINT cumulative losses / ISW estimated equipment (%)")
    plt.xlabel("Platform class"); plt.ylabel("% of estimated equipment")
    plt.tight_layout()
    plot_path = os.path.join(DIRS["plots"], f"losses_vs_isw_pct_{RUN_ID}.png")
    plt.savefig(plot_path, dpi=160); plt.close()
    log(f"Saved plot: {plot_path}")
else:
    log("Skipping losses_vs_isw chart (no ISW or daily data).")


[2025-09-08T14:57:03Z] ISW file not found at /content/drive/MyDrive/osint_mvp/ref/isw_force_snapshot_2025-09-08.csv. Please add it, then rerun this block.
[2025-09-08T14:57:03Z] ISW snapshot not available or empty — comparison will be skipped.
[2025-09-08T14:57:03Z] Skipping losses_vs_isw chart (no ISW or daily data).


/tmp/ipython-input-1192368844.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat(timespec='seconds') + "Z"
/tmp/ipython-input-1192368844.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat(timespec='seconds') + "Z"
/tmp/ipython-input-1192368844.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat(timespec='seconds') + "Z"


In [ ]:
import os, pandas as pd
from datetime import date

run = RUN_ID
paths = {
    "ws_silver": os.path.join(DIRS["staged"], f"warspotting_norm_{run}.csv"),
    "merged":    os.path.join(DIRS["out"],    f"merged_losses_{run}.csv"),
    "daily":     os.path.join(DIRS["out"],    f"merged_daily_class_{run}.csv"),
    "manifest":  os.path.join(DIRS["out"],    f"README_{run}.txt"),
}
for k,p in paths.items():
    print(f"{k}: {p} {'✓' if os.path.exists(p) else '✗'}")
    if p.endswith(".csv") and os.path.exists(p):
        df = pd.read_csv(p, parse_dates=["date"])
        print("  rows:", len(df), "date:", df['date'].min().date(), "→", df['date'].max().date())

print("\nPlots:")
for fn in sorted(os.listdir(DIRS["plots"])):
    if run in fn and fn.endswith(".png"):
        print("  -", os.path.join(DIRS["plots"], fn))


ws_silver: /content/drive/MyDrive/osint_mvp/staged/warspotting_norm_2025-09-08.csv ✓
  rows: 493 date: 2025-06-01 → 2025-09-07
merged: /content/drive/MyDrive/osint_mvp/out/merged_losses_2025-09-08.csv ✓
  rows: 493 date: 2025-06-01 → 2025-09-07
daily: /content/drive/MyDrive/osint_mvp/out/merged_daily_class_2025-09-08.csv ✓
  rows: 294 date: 2025-06-01 → 2025-09-07
manifest: /content/drive/MyDrive/osint_mvp/out/README_2025-09-08.txt ✓

Plots:
  - /content/drive/MyDrive/osint_mvp/plots/cum_losses_IFV_2025-09-08.png
  - /content/drive/MyDrive/osint_mvp/plots/cum_losses_MBT_2025-09-08.png
  - /content/drive/MyDrive/osint_mvp/plots/cum_losses_MLRS_2025-09-08.png
  - /content/drive/MyDrive/osint_mvp/plots/cum_losses_SPG_2025-09-08.png
  - /content/drive/MyDrive/osint_mvp/plots/cum_losses_TowedArtillery_2025-09-08.png
  - /content/drive/MyDrive/osint_mvp/plots/cum_losses_Truck_2025-09-08.png
  - /content/drive/MyDrive/osint_mvp/plots/daily_last14_IFV_2025-09-08.png
  - /content/drive/MyDrive/